In [1]:
%matplotlib inline
import sys
sys.path.append('../PythonAPI')
sys.path.append('../PythonAPI/pycocotools')
sys.path.append('./')
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from cbas import CBAS
from pycocotools.coco import COCO
# from pycocotools.cbas import CBAS
import cbas_construction_utils as ccu
import sklearn as sk

### Load COCO

In [2]:
# initialize COCO api for instance annotations and category info
cbas80=CBAS('../annotations/{}.json'.format('cbas80'))

# Get category index so we can go from image ids to category names
idToName={}
for c in cbas80.dataset['categories']:
    idToName[str(c['id'])]=c['name']

print("idToName: ", len(idToName.items()), idToName)
# base set is cbas34, and holdout set is cbas80-cbas34
# holdout is cbas80 - cbas34

loading annotations into memory...
Done (t=0.35s)
creating index...
index created!
idToName:  80 {'1': 'person', '2': 'bicycle', '3': 'car', '4': 'motorcycle', '5': 'airplane', '6': 'bus', '7': 'train', '8': 'truck', '9': 'boat', '10': 'traffic light', '11': 'fire hydrant', '13': 'stop sign', '14': 'parking meter', '15': 'bench', '16': 'bird', '17': 'cat', '18': 'dog', '19': 'horse', '20': 'sheep', '21': 'cow', '22': 'elephant', '23': 'bear', '24': 'zebra', '25': 'giraffe', '27': 'backpack', '28': 'umbrella', '31': 'handbag', '32': 'tie', '33': 'suitcase', '34': 'frisbee', '35': 'skis', '36': 'snowboard', '37': 'sports ball', '38': 'kite', '39': 'baseball bat', '40': 'baseball glove', '41': 'skateboard', '42': 'surfboard', '43': 'tennis racket', '44': 'bottle', '46': 'wine glass', '47': 'cup', '48': 'fork', '49': 'knife', '50': 'spoon', '51': 'bowl', '52': 'banana', '53': 'apple', '54': 'sandwich', '55': 'orange', '56': 'broccoli', '57': 'carrot', '58': 'hot dog', '59': 'pizza', '60': 

### Load cbas_34 base and holdout (AKA "target" classes) sets: 

In [3]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

transform = transforms.Compose([
#     transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

base_train = torchvision.datasets.ImageFolder(root='../images/cbas34_train', transform=transform)
base_train_loader = torch.utils.data.DataLoader(base_train, batch_size=4, shuffle=True, num_workers=4)

base_valid = torchvision.datasets.ImageFolder(root='../images/cbas34_val', transform=transform)
base_valid_loader = torch.utils.data.DataLoader(base_valid, batch_size=4, shuffle=False, num_workers=2)

# get index for curriculum sampling
id2idx = {}
for i,img in enumerate(base_train.imgs):
    img_id_str = img[0].split('/')[4].split('.')[0]
    id2idx[img_id_str] = i


In [4]:
print(base_train.__dict__.keys())
print(base_train.imgs[3])
print("Base classes ({} total): ".format(len(base_train.classes)))
print(base_train.classes)

print("base_train size: ", base_train.__len__())
print("steps per epoch: ", int(base_train.__len__() / 4))

dict_keys(['root', 'imgs', 'classes', 'class_to_idx', 'transform', 'target_transform', 'loader'])
('../images/cbas34_train/airplane/1363089.jpg', 0)
Base classes (34 total): 
['airplane', 'backpack', 'banana', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'car', 'carrot', 'chair', 'clock', 'cow', 'cup', 'donut', 'fork', 'handbag', 'horse', 'kite', 'knife', 'person', 'pottedplant', 'sheep', 'sink', 'skateboard', 'spoon', 'surfboard', 'tennisracket', 'trafficlight', 'truck', 'umbrella', 'vase']
base_train size:  51000
steps per epoch:  12750


### Find the holdout categories: 

In [5]:
# all_categories = COCO.loadCats(COCO.getCatIds())
# nms=[cat['name'] for cat in all_categories]
# print('COCO categories: \n{}\n'.format(' '.join(nms)))

# Get all categories: 


### Train network that learns to predict images from base_set: 

In [6]:
# Can get rid of this now that we have pytorch-classifcation/: 

# import torch.nn as nn

# __all__ = ['alexnet3232']


# class AlexNet3232(nn.Module):

#     def __init__(self, num_classes=34):
#         super(AlexNet3232, self).__init__()
#         self.features = nn.Sequential(
#             nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=5),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(64, 192, kernel_size=5, padding=2),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(192, 384, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(384, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#         )
#         self.classifier = nn.Linear(256, num_classes)

#     def forward(self, x):
#         x = self.features(x)
#         x = x.view(x.size(0), -1)
#         x = self.classifier(x)
#         return x


# def alexnet3232(**kwargs):
#     r"""AlexNet model architecture from the
#     `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
#     """
#     model = AlexNet(**kwargs)
#     return model

In [7]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

# num_classes_base = 34
# num_classes_holdout = 34

# Load model
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 34)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out


# model = LeNet()
# model = AlexNet3232()

### Load Trained Model: 

In [8]:
# Can get rid of this cell now that we can train using pytorch-classifcation/cbas.py: 

# import torch.optim as optim

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# epochs = 20
# print_every = 100
# steps_per_epoch = int(base_train.__len__() / 4)

# for epoch in range(epochs):  # loop over the dataset multiple times

#     running_loss = 0.0
#     for i, data in enumerate(base_train_loader, 0):
#         # get the inputs
#         inputs, labels = data

#         # wrap them in Variable
#         inputs, labels = Variable(inputs), Variable(labels)

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.data[0]
#         if i % print_every == (print_every-1):    # print every 2000 mini-batches
#             print('epoch[%d/%d, %5d/%d], loss: %.3f' %
#                   (epoch + 1, epochs, i + 1, steps_per_epoch, running_loss / print_every))
#             running_loss = 0.0

# print('Finished Training')

In [9]:
# Can get rid of this cell now that we can train using pytorch-classifcation/cbas.py: 
# import os 

# # Save model trained on cbas-LS base set:
# weights_dir = './weights/'
# if not os.path.exists(weights_dir):
#     os.makedirs(weights_dir)
# # torch.save(model, os.path.join(weights_dir, 'alexnet_cbas34_baseset_20_epochs.pt'))

In [53]:
import sys
sys.path.append("./pytorch-classification")
sys.path.append("./pytorch-classification/models")
sys.path.append("./pytorch-classification/models/cifar/")
import os
import cbas
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import models.cifar as models
from collections import OrderedDict

def load_model(path, model=None): 
    gpu_id = "0"
    os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id
    use_cuda = torch.cuda.is_available()
    print("use_cuda: ", use_cuda)
    
    if model is None: 
        model = models.alexnet(num_classes=34)

    if use_cuda:
        print("Using GPU")
        model = torch.nn.DataParallel(model).cuda()
        model = model.cuda()
        cudnn.benchmark = True
        checkpoint = torch.load(path)
        state_dict = checkpoint['state_dict']
    else: 
        print("Using CPU (no GPU)")
        checkpoint = torch.load(path, map_location=lambda storage, loc: storage)
        state_dict = checkpoint['state_dict']
        trained_on_gpu = list(state_dict.keys())[0].startswith('module.')
        if trained_on_gpu:
            new_state_dict = OrderedDict()
            for k,v in state_dict.items():
                name = k[7:] # remove module.
                new_state_dict[name] = v
            state_dict = new_state_dict
        else: 
            checkpoint = torch.load(path)
            state_dict = checkpoint['state_dict']
                
    
#     print(checkpoint.keys())
    print("state_dict: ", checkpoint['state_dict'].keys())
    model.load_state_dict(state_dict)
    return model, use_cuda


In [54]:
# Grab weights from my google drive: https://drive.google.com/drive/u/0/folders/1BFYG5soBG6vBV7RC00NjadPYdCapWLgb
# (use your berkeley google account)
model, use_cuda = load_model('./pytorch-classification/alexnet_300epochs/model_best.pth.tar')
print(use_cuda, model)

use_cuda:  False
Using CPU (no GPU)
state_dict:  odict_keys(['module.features.0.weight', 'module.features.0.bias', 'module.features.3.weight', 'module.features.3.bias', 'module.features.6.weight', 'module.features.6.bias', 'module.features.8.weight', 'module.features.8.bias', 'module.features.10.weight', 'module.features.10.bias', 'module.classifier.weight', 'module.classifier.bias'])
False AlexNet(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(5, 5))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d (64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d (192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d (384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d (256, 256,

In [104]:
import numpy as np 
from PIL import Image

class FeatureExtractor(object):
    
    def __init__(self, model=None, embed_layer=None, embed_size=256, transform=None):
        self.embed_size = embed_size
        self.transforms = transform
        
        if model is None: 
            self.model = models.alexnet(pretrained=True)
            self.embed_layer = self.model.features
        else: 
            if embed_layer is None: 
                raise ValueError("Need to specify embed_layer if you pass in a model to FeatureExtractor!")
            self.model = model
            self.embed_layer = embed_layer
        
        self.cuda = torch.cuda.is_available()
        if self.cuda:
            self.model.cuda()
            
        # Set model to eval mode so any train-specific things like dropout, etc. don't run:
        self.model.eval()
    
    def embed(self, img):
        """
        project a PIL image into embedded feature space, and return that vector as an np array
        """
        # Work arround issue of some of the train images not being RGB, (they won't go through the CNN): 
        if img.mode != "RGB":
            print("Hack: converting image to RGB before embedding w/ CNN...")
            rgbimg = Image.new("RGB", img.size)
            rgbimg.paste(img)
            img = rgbimg

        a = self.transforms(img)
        image = Variable(a)
        image = image.unsqueeze(0)
        if self.cuda: image.cuda()
        #print(image.size())
        
        embedding = torch.zeros(self.embed_size)
        def copy_embedding(m, i, o):
            if len(o.size()) > 2:
                o = o.view(o.size(0), -1)
            embedding.copy_(o.data)
            
        h = self.embed_layer.register_forward_hook(copy_embedding)
        h_x = self.model(image)
        h.remove()
        return embedding.numpy()
    

In [105]:
# import os 

# # Load model pre-trained on cbas-LS base set: 
# weights_dir = './weights/'
# if not os.path.exists(weights_dir):
#     os.makedirs(weights_dir)

# model = torch.load(os.path.join(weights_dir, 'alexnet_cbas34_baseset_20_epochs.pt'))

In [106]:
if use_cuda:
    embedder = FeatureExtractor(model=model, transform=transform, embed_layer=model.module.features)
else:
    embedder = FeatureExtractor(model=model, transform=transform, embed_layer=model.features)

In [107]:

# get some random training images
dataiter = iter(base_train_loader)
images, labels = dataiter.next()

# Test embedding a couple of images. For Alexnet (32x32 images) this should be a 256-d vector: 
img = Image.open('../images/cbas34_train/airplane/156356.jpg')
emb = embedder.embed(img)
print(emb.shape, emb)

img = Image.open('../images/cbas34_train/cow/69943.jpg')
emb = embedder.embed(img)
print(emb.shape, emb)

(256,) [  2.60561407e-01   0.00000000e+00   1.05016306e-01   1.45645487e+00
   6.41261041e-01   0.00000000e+00   0.00000000e+00   7.01496005e-01
   6.56437576e-01   1.20135117e+00   3.96289885e-01   0.00000000e+00
   7.33434111e-02   0.00000000e+00   3.52962971e-01   1.93643004e-01
   1.22039044e+00   1.04992640e+00   6.91160321e-01   3.86048675e-01
   0.00000000e+00   0.00000000e+00   1.83523893e-02   3.14123333e-02
   6.47000000e-02   2.17227265e-01   8.42344835e-02   7.58793414e-01
   4.54706043e-01   0.00000000e+00   9.94468108e-03   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   5.26395068e-02
   5.14534295e-01   3.08245689e-01   5.23454666e-01   7.32052147e-01
   7.76769996e-01   6.68430030e-01   7.16127977e-02   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   8.88239816e-02
   0.00000000e+00   1.84420025e+00   4.97282177e-01   8.67667079e-01
   0.00000000e+00   0.00000000e+00   9.91738796e-01   1.03220630e+00
   6.44936204e-01   3.83864

/home/gbiamby/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
/home/gbiamby/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.


### Check Embedding Results
Now let's compute cosine similarities for one airplane image against all airplanes, and then similarities for airplane against several other non-airplane categories. The average cosine similarity for airplane vs. airplane should be higher than that for airplane-vs-non-airplanes, if we are computing good embeddings. 

In [108]:
import glob 
from sklearn.metrics.pairwise import cosine_similarity
np.random.seed(55)

def get_cos_similarities(base_img_path, other_image_paths):
    img = Image.open(base_img_path)
    # Work arround issue of some of the train images not being RGB, (they won't go through the CNN): 
    if img.mode != "RGB":
        print("Hack: converting {} to rgb...".format(base_img_path))
        rgbimg = Image.new("RGB", img.size)
        rgbimg.paste(img)
        img = rgbimg
    base_embedding = embedder.embed(img)
    similarities = np.zeros((len(other_image_paths)))
    
    for idx,img_path in enumerate(other_image_paths):
        #print("idx: ", idx, img_path)
        embedding = None
        img = Image.open(img_path)
        try:
            embedding = embedder.embed(img)
        except:
            print("error on image #", idx)
        if embedding is not None: 
#             cos_sim = cosine_similarity(base_embedding, embedding)
            cos = nn.CosineSimilarity(dim=1, eps=1e-6)
            if use_cuda:
                cos_sim = cos(torch.Tensor(base_embedding).cuda().unsqueeze(0), torch.Tensor(embedding).cuda().unsqueeze(0))
            else: 
                cos_sim = cos(torch.Tensor(base_embedding).unsqueeze(0), torch.Tensor(embedding).unsqueeze(0))
            similarities[idx] = cos_sim.cpu().numpy()
    return similarities

def avg_category_similarities(dataset_path, cat1, cat2, sample_size=50):
    """
    Compute avereage similarities between all imgs in categories cat1 an cat2
    """
    cat1_imgs = glob.glob(dataset_path.format(cat1)+'*.jpg')
    cat2_imgs = glob.glob(dataset_path.format(cat2)+'*.jpg')
    # Sample a limited size to reduce compute time: 
    size1, size2 = min(len(cat1_imgs), sample_size), min(len(cat2_imgs), sample_size)
    cat1_imgs = np.random.choice(cat1_imgs, size=size1)
    cat2_imgs = np.random.choice(cat2_imgs, size=size2)
    sims = np.zeros((size1, size2))
    #print("similarites shape: ", sims.shape)
    for i,img1 in enumerate(cat1_imgs):
        sims[i,:] = get_cos_similarities(img1, cat2_imgs).T
        pass
    return sims


base_cat = "airplane"
other_cats = ["cow", "banana", "sink", "carrot", "umbrella"]

print("Testing intra-category cosine similarities for category: ", base_cat)
intra_cat_sims = avg_category_similarities('../images/cbas34_train/{}/', base_cat, base_cat)
print(
    "average '{}' intra-category similarity: {}\n".format(
        base_cat
        , np.average(intra_cat_sims)
    ))

for other_cat in other_cats: 
    print("Testing inter-category cosine similarities for '{}' vs '{}'..."\
          .format(base_cat, other_cat))
    inter_cat_sims = avg_category_similarities(
        '../images/cbas34_train/{}/', base_cat, other_cat
    )
    print("average {} cos-similarity vs '{}': {}\n".format(
        base_cat, other_cat, np.average(inter_cat_sims)
    ))


## NOTE: the error messages here are mostly due to some of the training 
## images in cbas-34 being 1 channel instead of 3. We should regenerate the 
## cbas set to be 3-channel to fix this. 

Testing intra-category cosine similarities for category:  airplane


/home/gbiamby/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
/home/gbiamby/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.


average 'airplane' intra-category similarity: 0.5330798383712768

Testing inter-category cosine similarities for 'airplane' vs 'cow'...
average airplane cos-similarity vs 'cow': 0.43208702360391615

Testing inter-category cosine similarities for 'airplane' vs 'banana'...
average airplane cos-similarity vs 'banana': 0.35064838972687723

Testing inter-category cosine similarities for 'airplane' vs 'sink'...
average airplane cos-similarity vs 'sink': 0.39051078941822054

Testing inter-category cosine similarities for 'airplane' vs 'carrot'...
Hack: converting ../images/cbas34_train/airplane/156497.jpg to rgb...
average airplane cos-similarity vs 'carrot': 0.349950625705719

Testing inter-category cosine similarities for 'airplane' vs 'umbrella'...
Hack: converting ../images/cbas34_train/airplane/247539.jpg to rgb...
average airplane cos-similarity vs 'umbrella': 0.44700413351655005



In [102]:
base_cat = "banana"
other_cats = ["cow", "airplane", "sink", "carrot", "umbrella"]


print("Testing intra-category cosine similarities for category: ", base_cat)
intra_cat_sims = avg_category_similarities('../images/cbas34_train/{}/', base_cat, base_cat)
print(
    "average '{}' intra-category similarity: {}\n".format(
        base_cat
        , np.average(intra_cat_sims)
    ))

for other_cat in other_cats: 
    print("Testing inter-category cosine similarities for '{}' vs '{}'..."\
          .format(base_cat, other_cat))
    inter_cat_sims = avg_category_similarities(
        '../images/cbas34_train/{}/', base_cat, other_cat
    )
    print("average {} cos-similarity vs '{}': {}\n".format(
        base_cat, other_cat, np.average(inter_cat_sims)
    ))

Testing intra-category cosine similarities for category:  banana


/home/gbiamby/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
/home/gbiamby/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.


average 'banana' intra-category similarity: 0.5884141979694366

Testing inter-category cosine similarities for 'banana' vs 'cow'...
average banana cos-similarity vs 'cow': 0.40151104258298875

Testing inter-category cosine similarities for 'banana' vs 'airplane'...
average banana cos-similarity vs 'airplane': 0.35424724360108373

Testing inter-category cosine similarities for 'banana' vs 'sink'...
average banana cos-similarity vs 'sink': 0.4279528742313385

Testing inter-category cosine similarities for 'banana' vs 'carrot'...
average banana cos-similarity vs 'carrot': 0.5344849278271199

Testing inter-category cosine similarities for 'banana' vs 'umbrella'...
average banana cos-similarity vs 'umbrella': 0.41140607810020446



### What happens when we try embedding images from classes in the holdout set?

In the preceding cells we did a sanity check to verify that embeddings made some kind of sense, but those were all embeddings the network was trained on. What about when we embed images for classes the network was not trained on? Let's try it out by doing similar comparisons for categories in the cbas-LS set (i.e., holdout set). 

In [38]:
base_cat = "mouse"
other_cats = ["orange", "suitcase", "toilet", "elephant", "giraffe"]

print("Testing intra-category cosine similarities for category: ", base_cat)
intra_cat_sims = avg_category_similarities('../images/cbasLS_train/{}/', base_cat, base_cat)
print(
    "average '{}' intra-category similarity: {}\n".format(
        base_cat
        , np.average(intra_cat_sims)
    ))

for other_cat in other_cats: 
    print("Testing inter-category cosine similarities for '{}' vs '{}'..."\
          .format(base_cat, other_cat))
    inter_cat_sims = avg_category_similarities(
        '../images/cbasLS_train/{}/', base_cat, other_cat
    )
    print("average {} cos-similarity vs '{}': {}\n".format(
        base_cat, other_cat, np.average(inter_cat_sims)
    ))


Testing intra-category cosine similarities for category:  mouse


/home/gbiamby/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.


average 'mouse' intra-category similarity: 0.5026432198226451

Testing inter-category cosine similarities for 'mouse' vs 'orange'...
average mouse cos-similarity vs 'orange': 0.45833721430301666

Testing inter-category cosine similarities for 'mouse' vs 'suitcase'...
average mouse cos-similarity vs 'suitcase': 0.45220311656594275

Testing inter-category cosine similarities for 'mouse' vs 'toilet'...
average mouse cos-similarity vs 'toilet': 0.4934432402729988

Testing inter-category cosine similarities for 'mouse' vs 'elephant'...
average mouse cos-similarity vs 'elephant': 0.38700988946557047

Testing inter-category cosine similarities for 'mouse' vs 'giraffe'...
average mouse cos-similarity vs 'giraffe': 0.398318815779686



In [39]:
base_cat = "orange"
other_cats = ["hotdog", "suitcase", "laptop", "scissors", "giraffe"]

print("Testing intra-category cosine similarities for category: ", base_cat)
intra_cat_sims = avg_category_similarities('../images/cbasLS_train/{}/', base_cat, base_cat)
print(
    "average '{}' intra-category similarity: {}\n".format(
        base_cat
        , np.average(intra_cat_sims)
    ))

for other_cat in other_cats: 
    print("Testing inter-category cosine similarities for '{}' vs '{}'..."\
          .format(base_cat, other_cat))
    inter_cat_sims = avg_category_similarities(
        '../images/cbasLS_train/{}/', base_cat, other_cat
    )
    print("average {} cos-similarity vs '{}': {}\n".format(
        base_cat, other_cat, np.average(inter_cat_sims)
    ))

Testing intra-category cosine similarities for category:  orange


/home/gbiamby/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.


average 'orange' intra-category similarity: 0.6251163078546524

Testing inter-category cosine similarities for 'orange' vs 'hotdog'...
average orange cos-similarity vs 'hotdog': 0.5699484491109849

Testing inter-category cosine similarities for 'orange' vs 'suitcase'...
average orange cos-similarity vs 'suitcase': 0.37430530357956887

Testing inter-category cosine similarities for 'orange' vs 'laptop'...
average orange cos-similarity vs 'laptop': 0.38799188279509544

Testing inter-category cosine similarities for 'orange' vs 'scissors'...
average orange cos-similarity vs 'scissors': 0.4093845841884613

Testing inter-category cosine similarities for 'orange' vs 'giraffe'...
average orange cos-similarity vs 'giraffe': 0.37873646223545077



In [40]:
base_cat = "pizza"
other_cats = ["hotdog", "remote", "laptop", "stopsign", "giraffe"]

print("Testing intra-category cosine similarities for category: ", base_cat)
intra_cat_sims = avg_category_similarities('../images/cbasLS_train/{}/', base_cat, base_cat)
print(
    "average '{}' intra-category similarity: {}\n".format(
        base_cat
        , np.average(intra_cat_sims)
    ))

for other_cat in other_cats: 
    print("Testing inter-category cosine similarities for '{}' vs '{}'..."\
          .format(base_cat, other_cat))
    inter_cat_sims = avg_category_similarities(
        '../images/cbasLS_train/{}/', base_cat, other_cat
    )
    print("average {} cos-similarity vs '{}': {}\n".format(
        base_cat, other_cat, np.average(inter_cat_sims)
    ))

Testing intra-category cosine similarities for category:  pizza


/home/gbiamby/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.


average 'pizza' intra-category similarity: 0.5749508616626262

Testing inter-category cosine similarities for 'pizza' vs 'hotdog'...
average pizza cos-similarity vs 'hotdog': 0.5535113266468048

Testing inter-category cosine similarities for 'pizza' vs 'remote'...
average pizza cos-similarity vs 'remote': 0.4865093769550323

Testing inter-category cosine similarities for 'pizza' vs 'laptop'...
average pizza cos-similarity vs 'laptop': 0.48646524358391763

Testing inter-category cosine similarities for 'pizza' vs 'stopsign'...
average pizza cos-similarity vs 'stopsign': 0.4035489796221256

Testing inter-category cosine similarities for 'pizza' vs 'giraffe'...
average pizza cos-similarity vs 'giraffe': 0.39467512319087983



### Now Try Few-Shot Classification

In [60]:
import sklearn as sk
from sklearn.neighbors import KNeighborsClassifier


In [127]:

# https://xkcd.com/221/ 
np.random.seed(4)

def get_joint_categories():
    joint_cats = []
    base_cats   = glob.glob('../images/cbas34_train/*/')
    holdout_cats = glob.glob('../images/cbasLS_train/*/')
    joint_cats.extend(base_cats)
    joint_cats.extend(holdout_cats)
    return joint_cats
    
# For each category in cbasLS, choose N examples from cbasLS train set:
def get_holdout_train_examples(n):
    """
    Gets a list of image paths from cbasLS train set. List contains n 
    randomly sampled images from each category in cbasLS
    """
    ls_path = '../images/cbasLS_train/'
    cats = glob.glob(ls_path + '*/')
    train_example_paths = []
    #print(len(cats), cats)
    for cat_path in cats:
        imgs = glob.glob("{}*.jpg".format(cat_path))
        train_example_paths.extend(np.random.choice(imgs, replace=False, size=n))
    return train_example_paths 

def get_embeddings(img_paths):
    """
    input: is list of image paths.
    output: list of tuples of form: (category, id, path, vec. embedding)
    """
    return [
        (os.path.dirname(p).split('/')[-1]
         , os.path.basename(p).replace('.jpg','')
         , p
         , embedder.embed(Image.open(p))
        ) for p in img_paths
    ]

# Get embeddings for images in holdout validation set: 
def get_ls_valid_embeddings():
    """
    output: list of tuples of form: (category, id, path, vec. embedding)
    """
    valid_paths = glob.glob('../images/cbasLS_val/*/*.jpg')
    print("Found {} validation images in cbasLS_val. Getting embeddings...".format(len(valid_paths)))
    embeddings = get_embeddings(valid_paths)
    return embeddings

def lowshot_valid_acc(X_valid, y_valid, y_pred):
    """
    Returns validation accuracy 
    """
    if not ((len(X_valid) == len(y_valid)) and (len(X_valid)==len(y_pred))):
        raise ValueError(
            "Lengths must be the same for: X_valid:{}, y_valid:{}, y_pred:{}".format(
            len(X_valid), len(y_valid), len(y_pred)
        ))
    total, total_acc = len(X_valid), 0
    for i, x_val in enumerate(X_valid):
        total_acc += 1 if y_valid[i]==y_pred[i] else 0
    val_acc = float(total_acc) / float(total)
    return val_acc



#### Embed the images in the cbas low-shot valid set

In [129]:
def lowshot_fit_and_predict(N, target_set="holdout"):
    """
    Trains knn classifier with N training examples from each category in cbasLS, 
    and returns validation accuracy on entire cbasLS_val set
    inputs: 
        N: num. training examples per cbasLS_train category 
        target_set: (str) indicates which set of categories we train classifier with. 
                    Shuold either be: 'holdout', or 'joint'
    """
    if target_set!="holdout":
        raise NotImplementedError("Only support for target_set='holdout' is available rn.")
    train_paths = get_holdout_train_examples(N)
    train_samples = get_embeddings(train_paths)
    # print(train_samples)
    #joint_cats = get_joint_categories()
    #print("Found {} total joint (base + holdout) cats".format(len(joint_cats)))

    # Fit a knn classifier to those N training examples: 
    X_train = [vec for cat,id,p,vec in train_samples]
    y_train = [cat for cat,id,p,vec in train_samples]
    knn = KNeighborsClassifier(n_neighbors=N)
    knn.fit(X_train, y_train)

    ls_valid_set = get_ls_valid_embeddings()
    #print(ls_valid_set[0])
    X_valid = [vec for cat,id,p,vec in ls_valid_set]
    y_valid = [cat for cat,id,p,vec in ls_valid_set]
    y_pred = knn.predict(X_valid)

    # Val acc: 
    return lowshot_valid_acc(X_valid, y_valid, y_pred)

N_vals = [1, 2, 5, 10, 20]

for N in N_vals: 
    print("Training lowshot predictor for N={}".format(N))
    print("validation acc.: {}".format(lowshot_fit_and_predict(N)))

Training lowshot predictor for N=1
Found 8330 validation images in cbasLS_val. Getting embeddings...


/home/gbiamby/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
/home/gbiamby/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.


Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
validation acc.: 0.09015606242496998
Training lowshot predictor for N=2
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Found 8330 validation images in cbasLS_val. Getting embeddings...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
validation acc.: 0.08151260504201681
Training lowshot predictor for N=5
Hack: converting {} to rgb...
Found 8330 validation images in cbasLS_val. Getting embeddings...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack: converting {} to rgb...
Hack